## Croping image based on the co-ordinate file with black background

In [33]:
import cv2
import numpy as np
from PIL import Image
import os

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            self.process_image(image_path, file_path)

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Convert the masked image to a PIL Image
        cropped_image = Image.fromarray(masked_image)

        # Save the cropped image with the name of the coordinate file
        output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
        os.makedirs(output_folder, exist_ok=True)
        output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
        output_path = os.path.join(output_folder, output_filename)
        cropped_image.save(output_path)
        print(f"Cropped image saved to: {output_path}")

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\140_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b103_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b104_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b105_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b106_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b107_crop

KeyboardInterrupt: 

## following code is not taking the backgrond which is in black


In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            self.process_image(image_path, file_path)

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(original_image, dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find the bounding box of the masked region
        _, thresh = cv2.threshold(cv2.cvtColor(masked_image, cv2.COLOR_RGB2GRAY), 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        x, y, w, h = cv2.boundingRect(contours[0])

        # Crop the region of interest from the original image
        cropped_image = original_image[y:y+h, x:x+w]

        # Convert the cropped image to a PIL Image
        cropped_image_pil = Image.fromarray(cropped_image)

        # Save the cropped image with the name of the coordinate file
        output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
        os.makedirs(output_folder, exist_ok=True)
        output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
        output_path = os.path.join(output_folder, output_filename)
        cropped_image_pil.save(output_path)
        print(f"Cropped image saved to: {output_path}")

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\140_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b103_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b104_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b105_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b106_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b107_crop

## follwoing code is also cropping the image without black backround

In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            self.process_image(image_path, file_path)

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(original_image, dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the convex hull of the contour
        hull = cv2.convexHull(contours[0])

        # Draw the convex hull on the mask
        cv2.fillConvexPoly(mask, hull, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find the bounding box of the masked region
        x, y, w, h = cv2.boundingRect(hull)

        # Crop the region of interest from the original image
        cropped_image = original_image[y:y+h, x:x+w]

        # Convert the cropped image to a PIL Image
        cropped_image_pil = Image.fromarray(cropped_image)

        # Save the cropped image with the name of the coordinate file
        output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
        os.makedirs(output_folder, exist_ok=True)
        output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
        output_path = os.path.join(output_folder, output_filename)
        cropped_image_pil.save(output_path)
        print(f"Cropped image saved to: {output_path}")

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\140_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b103_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b104_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b105_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b106_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b107_crop

## Cropping image without background but it is taking some extra image which is not region of interest

In [7]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            self.process_image(image_path, file_path)

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(original_image, dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(contours[0])

        # Crop the region of interest from the original image
        cropped_image = original_image[y:y+h, x:x+w]

        # Convert the cropped image to a PIL Image
        cropped_image_pil = Image.fromarray(cropped_image)

        # Save the cropped image with the name of the coordinate file
        output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
        os.makedirs(output_folder, exist_ok=True)
        output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
        output_path = os.path.join(output_folder, output_filename)
        cropped_image_pil.save(output_path)
        print(f"Cropped image saved to: {output_path}")

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\140_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b103_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b104_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b105_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b106_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b107_crop

## following code is cropping the image and minimize the black part which is goood.

In [8]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            self.process_image(image_path, file_path)

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Find the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(contours[0])

        # Crop the region of interest from the original image
        cropped_image = masked_image[y:y+h, x:x+w]

        # Convert the cropped image to a PIL Image
        cropped_image_pil = Image.fromarray(cropped_image)

        # Save the cropped image with the name of the coordinate file
        output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
        os.makedirs(output_folder, exist_ok=True)
        output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
        output_path = os.path.join(output_folder, output_filename)
        cropped_image_pil.save(output_path)
        print(f"Cropped image saved to: {output_path}")

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\140_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b103_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b104_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b105_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b106_cropped.jpg
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b107_crop

## Following code are dedicated to  load the hugging face crackDetection model by using the code

In [5]:
!pip install transformers

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     --------------- ----------------------- 51.2/129.4 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 129.4/129.4 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   -- ------------------------------------- 0.6/8.4 MB 11.5 MB/s eta 0:00:01
   ----- ---------------------------------- 1.1/8.4 MB 12.0 MB/s eta 0:00:01
   ------- -------------------------------- 1.6/8.4 MB 11.7 MB/s eta 0:00:01
   ---------- ----------------------------- 2.2/8.4 MB 11.6 MB/s eta 0:00:01
   ------------ --------------------------- 2.7/8.4 MB 11.5 MB/s eta 0:00:01
   --------------- ------------------------ 3.3/8.4 MB 11.6 MB/s eta 0:00:01
   ----------------- ---------------------- 3.8/8.4 MB 11.5 MB/s eta 0:00:01
   -----------

In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-classification", model="Taki3d/CrackDetection")
result = pipe(r"C:\Users\abhishek\Desktop\216\Image\cropped_images\140_cropped.jpg")
print(result)

[{'label': 'noncrack', 'score': 0.986605703830719}, {'label': 'crack', 'score': 0.01339428499341011}]


In [12]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("Taki3d/CrackDetection")
model = AutoModelForImageClassification.from_pretrained("Taki3d/CrackDetection")

## following code is detecting the crack on each and every block

In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog
from transformers import pipeline

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")
        # Initialize the crack detection pipeline
        self.crack_detection_pipeline = pipeline("image-classification", model="Taki3d/CrackDetection")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            self.process_image(image_path, file_path)

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Find the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(contours[0])

        # Crop the region of interest from the original image
        cropped_image = masked_image[y:y+h, x:x+w]

        # Convert the cropped image to a PIL Image
        cropped_image_pil = Image.fromarray(cropped_image)

        # Save the cropped image with the name of the coordinate file
        output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
        os.makedirs(output_folder, exist_ok=True)
        output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
        output_path = os.path.join(output_folder, output_filename)
        cropped_image_pil.save(output_path)
        print(f"Cropped image saved to: {output_path}")

        # Apply crack detection on the cropped image
        crack_detection_result = self.crack_detection_pipeline(output_path)
        print("Crack detection result:", crack_detection_result)

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\140_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.986605703830719}, {'label': 'crack', 'score': 0.01339428499341011}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Crack detection result: [{'label': 'crack', 'score': 0.5316077470779419}, {'label': 'noncrack', 'score': 0.4683922529220581}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.8740609884262085}, {'label': 'crack', 'score': 0.1259389966726303}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.99482661485672}, {'label': 'crack', 'score': 0.005173325538635254}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Crack detection result: [{'label': 'nonc

## follwoing code is representing the highlihts

croppping image but note cropping as previous 

In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog
from transformers import pipeline

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")
        # Initialize the crack detection pipeline
        self.crack_detection_pipeline = pipeline("image-classification", model="Taki3d/CrackDetection")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Initialize a variable to store all outlines
        all_outlines = []

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            outlines = self.process_image(image_path, file_path)
            all_outlines.extend(outlines)

        # Draw all outlines on the original image
        original_image_with_outlines = self.draw_outlines(image_path, all_outlines)

        # Save the original image with outlines
        outlines_output_folder = os.path.join(os.path.dirname(image_path), "outlines")
        os.makedirs(outlines_output_folder, exist_ok=True)
        outlines_output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_with_outlines.jpg"
        outlines_output_path = os.path.join(outlines_output_folder, outlines_output_filename)
        cv2.imwrite(outlines_output_path, cv2.cvtColor(original_image_with_outlines, cv2.COLOR_RGB2BGR))
        print(f"Image with outlines saved to: {outlines_output_path}")

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Draw outlines on the original image
        for outline in roi_corners:
            cv2.polylines(original_image, [outline], isClosed=True, color=(0, 255, 0), thickness=2)

        # Convert the outlines to bounding rectangles and process crack detection
        for outline in roi_corners:
            # Find the bounding rectangle of the outline
            x, y, w, h = cv2.boundingRect(outline)

            # Crop the region of interest from the original image
            cropped_image = original_image[y:y+h, x:x+w]

            # Convert the cropped image to a PIL Image
            cropped_image_pil = Image.fromarray(cropped_image)

            # Save the cropped image with the name of the coordinate file
            output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cropped_image_pil.save(output_path)
            print(f"Cropped image saved to: {output_path}")

            # Apply crack detection on the cropped image
            crack_detection_result = self.crack_detection_pipeline(output_path)
            print("Crack detection result:", crack_detection_result)

        return roi_corners

    def draw_outlines(self, image_path, outlines):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image_with_outlines = original_image.copy()  # Create a copy for drawing outlines

        # Draw outlines on the original image
        for outline in outlines:
            cv2.polylines(original_image_with_outlines, [outline], isClosed=True, color=(0, 255, 0), thickness=2)

        return original_image_with_outlines

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\140_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9930025935173035}, {'label': 'crack', 'score': 0.006997427437454462}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9844534993171692}, {'label': 'crack', 'score': 0.015546501614153385}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9603212475776672}, {'label': 'crack', 'score': 0.03967871889472008}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9429178237915039}, {'label': 'crack', 'score': 0.057082127779722214}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Crack detection result: [{'label'

## following code is creating green outline for non crack and red outline for crack

In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog
from transformers import pipeline

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")
        # Initialize the crack detection pipeline
        self.crack_detection_pipeline = pipeline("image-classification", model="Taki3d/CrackDetection")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Initialize a variable to store all outlines
        all_outlines = []

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            outlines = self.process_image(image_path, file_path)
            all_outlines.extend(outlines)

        # Draw all outlines on the original image
        original_image_with_outlines = self.draw_outlines(image_path, all_outlines)

        # Save the image with outlines
        outlines_output_folder = os.path.join(os.path.dirname(image_path), "outlines")
        os.makedirs(outlines_output_folder, exist_ok=True)
        outlines_output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_with_outlines.jpg"
        outlines_output_path = os.path.join(outlines_output_folder, outlines_output_filename)
        cv2.imwrite(outlines_output_path, cv2.cvtColor(original_image_with_outlines, cv2.COLOR_RGB2BGR))
        print(f"Image with outlines saved to: {outlines_output_path}")

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Initialize a list to store outlines
        outlines = []

        # Iterate through each contour
        for contour in contours:
            # Find the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Crop the region of interest from the original image
            cropped_image = masked_image[y:y+h, x:x+w]

            # Convert the cropped image to a PIL Image
            cropped_image_pil = Image.fromarray(cropped_image)

            # Save the cropped image with the name of the coordinate file
            output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cropped_image_pil.save(output_path)
            print(f"Cropped image saved to: {output_path}")

            # Apply crack detection on the cropped image
            crack_detection_result = self.crack_detection_pipeline(output_path)
            print("Crack detection result:", crack_detection_result)

            # Check if crack score > 0.20
            if crack_detection_result[1]['score'] > 0.20:
                outlines.append((contour, (255, 0, 0)))  # Red outline for cracks
            # else:
            #     outlines.append((contour, (0, 255, 0)))  # Green outline for non-cracks
        
        return outlines

    def draw_outlines(self, image_path, outlines):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image_with_outlines = original_image.copy()  # Create a copy for drawing outlines

        # Draw outlines on the original image
        for outline, color in outlines:
            cv2.drawContours(original_image_with_outlines, [outline], -1, color, 2)

        return original_image_with_outlines

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


c:\Users\abhishek\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Crack detection result: [{'label': 'crack', 'score': 0.5316077470779419}, {'label': 'noncrack', 'score': 0.4683922529220581}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.8740609884262085}, {'label': 'crack', 'score': 0.1259389966726303}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.99482661485672}, {'label': 'crack', 'score': 0.005173325538635254}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9926577210426331}, {'label': 'crack', 'score': 0.007342291530221701}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Crack detection result: [{'label': '

## following code is also working fine with annotation of block and red outline

In [6]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog
from transformers import pipeline

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")
        # Initialize the crack detection pipeline
        self.crack_detection_pipeline = pipeline("image-classification", model="Taki3d/CrackDetection")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Initialize a variable to store all outlines
        all_outlines = []

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            outlines = self.process_image(image_path, file_path)
            all_outlines.extend(outlines)

        # Draw all outlines on the original image
        original_image_with_outlines = self.draw_outlines(image_path, all_outlines)

        # Save the image with outlines
        outlines_output_folder = os.path.join(os.path.dirname(image_path), "outlines")
        os.makedirs(outlines_output_folder, exist_ok=True)
        outlines_output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_with_outlines.jpg"
        outlines_output_path = os.path.join(outlines_output_folder, outlines_output_filename)
        cv2.imwrite(outlines_output_path, cv2.cvtColor(original_image_with_outlines, cv2.COLOR_RGB2BGR))
        print(f"Image with outlines saved to: {outlines_output_path}")

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Initialize a list to store outlines
        outlines = []

        # Iterate through each contour
        for contour in contours:
            # Find the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Crop the region of interest from the original image
            cropped_image = masked_image[y:y+h, x:x+w]

            # Convert the cropped image to a PIL Image
            cropped_image_pil = Image.fromarray(cropped_image)

            # Save the cropped image with the name of the coordinate file
            output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cropped_image_pil.save(output_path)
            print(f"Cropped image saved to: {output_path}")

            # Apply crack detection on the cropped image
            crack_detection_result = self.crack_detection_pipeline(output_path)
            print("Crack detection result:", crack_detection_result)

            # Check if crack score > 0.20
            if crack_detection_result[1]['score'] > 0.50:
                outlines.append((contour, (255, 0, 0), coordinates_file))  # Red outline for cracks
            # else:
            #     outlines.append((contour, (0, 255, 0), coordinates_file))  # Green outline for non-cracks
        
        return outlines

    def draw_outlines(self, image_path, outlines):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image_with_outlines = original_image.copy()  # Create a copy for drawing outlines

        # Draw outlines on the original image
        for outline, color, coordinate_file in outlines:
            cv2.drawContours(original_image_with_outlines, [outline], -1, color, 2)

            # Add coordinate file name as text annotation
            cv2.putText(original_image_with_outlines, os.path.splitext(os.path.basename(coordinate_file))[0],
                        (int(outline[:, :, 0].mean()), int(outline[:, :, 1].mean())),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        return original_image_with_outlines

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Crack detection result: [{'label': 'crack', 'score': 0.5316077470779419}, {'label': 'noncrack', 'score': 0.4683922529220581}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.8740609884262085}, {'label': 'crack', 'score': 0.1259389966726303}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.99482661485672}, {'label': 'crack', 'score': 0.005173325538635254}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9926577210426331}, {'label': 'crack', 'score': 0.007342291530221701}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Crack detection result: [{'label': 'n

## following code is giving the perfect result without number on the block

In [9]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog
from transformers import pipeline

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")
        # Initialize the crack detection pipeline
        self.crack_detection_pipeline = pipeline("image-classification", model="Taki3d/CrackDetection")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Initialize a variable to store all outlines
        all_outlines = []

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            outlines = self.process_image(image_path, file_path)
            all_outlines.extend(outlines)

        # Draw all outlines on the original image
        original_image_with_outlines = self.draw_outlines(image_path, all_outlines)

        # Save the image with outlines
        outlines_output_folder = os.path.join(os.path.dirname(image_path), "outlines")
        os.makedirs(outlines_output_folder, exist_ok=True)
        outlines_output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_with_outlines.jpg"
        outlines_output_path = os.path.join(outlines_output_folder, outlines_output_filename)
        cv2.imwrite(outlines_output_path, cv2.cvtColor(original_image_with_outlines, cv2.COLOR_RGB2BGR))
        print(f"Image with outlines saved to: {outlines_output_path}")

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Initialize a list to store outlines
        outlines = []

        # Iterate through each contour
        for contour in contours:
            # Find the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Crop the region of interest from the original image
            cropped_image = masked_image[y:y+h, x:x+w]

            # Convert the cropped image to a PIL Image
            cropped_image_pil = Image.fromarray(cropped_image)

            # Save the cropped image with the name of the coordinate file
            output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cropped_image_pil.save(output_path)
            print(f"Cropped image saved to: {output_path}")

            # Apply crack detection on the cropped image
            crack_detection_result = self.crack_detection_pipeline(output_path)
            print("Crack detection result:", crack_detection_result)

            # Check if crack score > 0.30
            for result in crack_detection_result:
                if result['label'] == 'crack' and result['score'] > 0.50:
                    outlines.append((contour, (255, 0, 0)))  # Red outline for high crack score
                    break  # Once we find the score for 'crack', we break the loop
        
        return outlines

    def draw_outlines(self, image_path, outlines):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image_with_outlines = original_image.copy()  # Create a copy for drawing outlines

        # Draw outlines on the original image
        for contour, color in outlines:
            cv2.drawContours(original_image_with_outlines, [contour], -1, color, 2)

        return original_image_with_outlines

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Crack detection result: [{'label': 'crack', 'score': 0.5316077470779419}, {'label': 'noncrack', 'score': 0.4683922529220581}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.8740609884262085}, {'label': 'crack', 'score': 0.1259389966726303}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.99482661485672}, {'label': 'crack', 'score': 0.005173325538635254}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9926577210426331}, {'label': 'crack', 'score': 0.007342291530221701}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Crack detection result: [{'label': 'n

In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog
from transformers import pipeline

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")
        # Initialize the crack detection pipeline
        self.crack_detection_pipeline = pipeline("image-classification", model="Taki3d/CrackDetection")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Initialize a variable to store all outlines
        all_outlines = []

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            outlines = self.process_image(image_path, file_path)
            all_outlines.extend(outlines)

        # Draw all outlines on the original image
        original_image_with_outlines = self.draw_outlines(image_path, all_outlines, coordinate_files)

        # Save the image with outlines
        outlines_output_folder = os.path.join(os.path.dirname(image_path), "outlines")
        os.makedirs(outlines_output_folder, exist_ok=True)
        outlines_output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_with_outlines.jpg"
        outlines_output_path = os.path.join(outlines_output_folder, outlines_output_filename)
        cv2.imwrite(outlines_output_path, cv2.cvtColor(original_image_with_outlines, cv2.COLOR_RGB2BGR))
        print(f"Image with outlines saved to: {outlines_output_path}")

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Initialize a list to store outlines
        outlines = []

        # Iterate through each contour
        for contour in contours:
            # Find the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Crop the region of interest from the original image
            cropped_image = masked_image[y:y+h, x:x+w]

            # Convert the cropped image to a PIL Image
            cropped_image_pil = Image.fromarray(cropped_image)

            # Save the cropped image with the name of the coordinate file
            output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cropped_image_pil.save(output_path)
            print(f"Cropped image saved to: {output_path}")

            # Apply crack detection on the cropped image
            crack_detection_result = self.crack_detection_pipeline(output_path)
            print("Crack detection result:", crack_detection_result)

            # Check if crack score > 0.30
            for result in crack_detection_result:
                if result['label'] == 'crack' and result['score'] > 0.30:
                    outlines.append((contour, (255, 0, 0), os.path.splitext(os.path.basename(coordinates_file))[0]))  # Red outline for high crack score
                    break  # Once we find the score for 'crack', we break the loop
        
        return outlines

    def draw_outlines(self, image_path, outlines, coordinate_files):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image_with_outlines = original_image.copy()  # Create a copy for drawing outlines

        # Draw outlines on the original image
        for contour, color, coord_file_name in outlines:
            cv2.drawContours(original_image_with_outlines, [contour], -1, color, 2)

            # Add coordinate file name as text annotation
            cv2.putText(original_image_with_outlines, coord_file_name,
                        (int(contour[:, :, 0].mean()), int(contour[:, :, 1].mean())),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

        return original_image_with_outlines

if __name__ == "__main__":
    root = tk.Tk()
    cropper = ImageCropper(root)
    cropper.select_image_and_coordinates()
    root.mainloop()


Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b1_cropped.jpg
Crack detection result: [{'label': 'crack', 'score': 0.5316077470779419}, {'label': 'noncrack', 'score': 0.4683922529220581}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b10_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.8740609884262085}, {'label': 'crack', 'score': 0.1259389966726303}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b100_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.99482661485672}, {'label': 'crack', 'score': 0.005173325538635254}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b101_cropped.jpg
Crack detection result: [{'label': 'noncrack', 'score': 0.9926577210426331}, {'label': 'crack', 'score': 0.007342291530221701}]
Cropped image saved to: C:/Users/abhishek/Desktop/216/Image\cropped_images\b102_cropped.jpg
Crack detection result: [{'label': 'n